In [1]:

import pymongo
import pprint
import pandas as pd
from pymongo import MongoClient
import json, os
import sqlite3
conn = sqlite3.connect('TMDB_Database')

In [2]:
def mongoimport(csv_path, dbname, collname, dburl='localhost',db_port=27017):
    client = MongoClient(dburl, db_port)
    db = client[dbname]
    coll = db[collname]
    data = pd.read_csv(csv_path, encoding='latin1')
    payload = json.loads(data.to_json(orient='records'))
    coll.remove()
    coll.insert(payload)
    return coll.count()


In [3]:
# -------PART 1--------------------
# Adding CSV files to dataframes.
#GROUP1
moviesfile = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_MOVIES.csv', encoding='latin1')
genresfile = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_GENRES.csv', encoding='latin1')
producersfile = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_PRODUCTIONS.csv', encoding='latin1')
movgenlink = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_GENRE_MOVIE.csv', encoding='latin1')
movprodlink = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_PRODUCTION_MOVIE.csv', encoding='latin1')

#GROUP2
usersfile = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_USERS.csv', encoding='latin1')
tweetsfile = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_TWEETS.csv', encoding='latin1')
tweetmovlink = pd.read_csv('C:/Users/ashwi/OneDrive/Documents/DMDD/moviesCSV/T_TWEET_MOVIE.csv', encoding='latin1')



In [4]:
#Inserting the  dataframes  into the physical database
moviesfile.to_sql('movies',con=conn,index=False, if_exists='replace')

producersfile.to_sql('producers',con=conn,index=False, if_exists='replace')

movprodlink.to_sql('movprod',con=conn,index=False, if_exists='replace')
usersfile.to_sql('users',con=conn,index=False, if_exists='replace')
tweetsfile.to_sql('tweets',con=conn,index=False, if_exists='replace')
tweetmovlink.to_sql('tweetmov',con=conn,index=False, if_exists='replace')

moviesfile

,MV_ID,MV_NAME,MV_RELDATE,MV_RUNTIME,MV_BUDGET,MV_RATING,MV_LINK,MV_DIRECTOR
0,5501,The Guilty,2018,85 min,207140,7.6,www.imdb.com/title/tt6742252/?ref_=ttls_li_tt,Gustav Möller
1,5502,Mission: Impossible - Fallout,2018,147 min,220159104,7.8,www.imdb.com/title/tt4912910/?ref_=ttls_li_tt,Christopher McQuarrie
2,5503,Searching,2018,102 min,26020957,7.7,www.imdb.com/title/tt7668870/?ref_=ttls_li_tt,Aneesh Chaganty
3,5504,A Star Is Born,2018,136 min,213097381,7.8,www.imdb.com/title/tt1517451/?ref_=ttls_li_tt,Bradley Cooper
4,5505,Spider-Man: Into the Spider-Verse,2018,117 min,187622827,8.6,www.imdb.com/title/tt4633694/?ref_=ttls_li_tt,Rodney Rothman
...,...,...,...,...,...,...,...,...
95,5596,Casual,2018,25 min,6349,7.6,www.imdb.com/title/tt4577466/?ref_=ttls_li_tt,NaN
96,5597,The Marvelous Mrs. Maisel,2018,57 min,39861,8.7,www.imdb.com/title/tt5788792/?ref_=ttls_li_tt,NaN
97,5598,Altered Carbon,2018,60 min,97285,8.2,www.imdb.com/title/tt2261227/?ref_=ttls_li_tt,NaN
98,5599,Barry,2018,30 min,15237,8.1,www.imdb.com/title/tt5348176/?ref_=ttls_li_tt,NaN


In [5]:

def run_query(query):
    return pd.read_sql(query,conn)
run_query('select * from sqlite_master where type = "table"')
import psycopg2
# conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
# df1= pd.read_sql('(SELECT "MV_ID","MV_NAME","MV_RELDATE","MV_RATING","MV_DIRECTOR" from movies ) A FULL OUTER JOIN (SELECT "GN_ID","MV_ID" from movgen) B on A."MV_ID"=B."MV_ID";',con=conn)

# DE-NORMALIZATION  AND COMBINING OF TABLES

movdf= run_query('SELECT movies.MV_ID,MV_NAME,MV_RELDATE,MV_RUNTIME,MV_RATING,MV_DIRECTOR,PC_NAME FROM movprod  LEFT JOIN  movies on movies.MV_ID=movprod.MV_ID LEFT JOIN producers on movprod.PC_ID=producers.PC_ID')
tweetdf= run_query('SELECT USR_ID,tweets.TW_ID,TW_DETAIL,TW_HASHTAG,MV_NAME from tweetmov LEFT JOIN tweets ON tweets.TW_ID=tweetmov.TW_ID LEFT JOIN movies ON tweetmov.MV_ID=movies.MV_ID')
tweetdf

# CONVERTING THE DE-NORMALIZED TABLES TO CSV FILES
movdf.to_csv('ONFMovies.csv',encoding='utf-8',index=False)
tweetdf.to_csv('ONFTweets.csv',encoding='utf-8',index=False)

In [6]:
# -------------------PART 2----------------
from pymongo import MongoClient



In [7]:
# ESTABLISHING MONGOCLIENT INSTANCE

# movpath= 'C:/Users/ashwi/PycharmProjects/dmddassignment3/ONFMovies.csv'
# dbname = 'moviedb'
# collname='movies'
# mongoimport(movpath,dbname,collname)
# movpath= 'C:/Users/ashwi/PycharmProjects/dmddassignment3/ONFTweets.csv'
# dbname = 'moviedb'
# collname='tweets'
# mongoimport(movpath,dbname,collname)
# 
# dbname.collname.find()
conn1 = pymongo.MongoClient()
conn1



MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [8]:
# specifying local host
MongoClient('localhost', 27017)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

In [9]:
# Defining the database

db = conn1['assignment3']
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'assignment3')

In [10]:
# DROPPING PRE-EXISTING TABLES

db.users.drop()
db.directors.drop()
db.movies.drop()
db.tweets.drop()
# Defining the collection
collection = db.movies
collection




Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'assignment3'), 'movies')

In [11]:
# READING CSV FILES INTO DATAFRAME
def load_csv(csv):
    p=os.path.join("data/", csv)
    print (p)
    data=pd.read_csv(p, encoding = "ISO-8859-1", engine='python')
    return data 
movies_csv=load_csv('C:/Users/ashwi/PycharmProjects/dmddassignment3/ONFMovies.csv')
movies_csv.head()


C:/Users/ashwi/PycharmProjects/dmddassignment3/ONFMovies.csv


,MV_ID,MV_NAME,MV_RELDATE,MV_RUNTIME,MV_RATING,MV_DIRECTOR,PC_NAME
0,5502,Mission: Impossible - Fallout,2018,147 min,7.8,Christopher McQuarrie,Paramount Pictures
1,5504,A Star Is Born,2018,136 min,7.8,Bradley Cooper,Warner Bros. Pictures
2,5507,You Were Never Really Here,2018,89 min,6.8,Lynne Ramsay,Amazon Studios
3,5508,A Quiet Place,2018,90 min,7.6,John Krasinski,Paramount Pictures
4,5509,The Miseducation of Cameron Post,2018,91 min,6.6,Desiree Akhavan,FilmRise


In [12]:
# The DATAFRAME is converted to JSON format and assigned to movies_json.

movies_json =json.loads(movies_csv.to_json(orient='records'))
movies_json[0]


{'MV_ID': 5502,
 'MV_NAME': 'Mission: Impossible - Fallout',
 'MV_RELDATE': 2018,
 'MV_RUNTIME': '147 min',
 'MV_RATING': 7.8,
 'MV_DIRECTOR': 'Christopher McQuarrie',
 'PC_NAME': 'Paramount Pictures'}

In [13]:
try:
    result = db.movies.drop()
    print ("testing movies drop")
except:
    pass



#inserting the movies_json dataset into the  MOVIES collection
collection.insert(movies_json)

testing movies drop


<ipython-input-13-734243a852e8>:10: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(movies_json)


[ObjectId('5e8e71db86757736aa93aaa7'),
 ObjectId('5e8e71db86757736aa93aaa8'),
 ObjectId('5e8e71db86757736aa93aaa9'),
 ObjectId('5e8e71db86757736aa93aaaa'),
 ObjectId('5e8e71db86757736aa93aaab'),
 ObjectId('5e8e71db86757736aa93aaac'),
 ObjectId('5e8e71db86757736aa93aaad'),
 ObjectId('5e8e71db86757736aa93aaae'),
 ObjectId('5e8e71db86757736aa93aaaf'),
 ObjectId('5e8e71db86757736aa93aab0'),
 ObjectId('5e8e71db86757736aa93aab1'),
 ObjectId('5e8e71db86757736aa93aab2'),
 ObjectId('5e8e71db86757736aa93aab3'),
 ObjectId('5e8e71db86757736aa93aab4'),
 ObjectId('5e8e71db86757736aa93aab5'),
 ObjectId('5e8e71db86757736aa93aab6'),
 ObjectId('5e8e71db86757736aa93aab7'),
 ObjectId('5e8e71db86757736aa93aab8'),
 ObjectId('5e8e71db86757736aa93aab9'),
 ObjectId('5e8e71db86757736aa93aaba'),
 ObjectId('5e8e71db86757736aa93aabb'),
 ObjectId('5e8e71db86757736aa93aabc'),
 ObjectId('5e8e71db86757736aa93aabd'),
 ObjectId('5e8e71db86757736aa93aabe'),
 ObjectId('5e8e71db86757736aa93aabf'),
 ObjectId('5e8e71db867577

In [14]:
# Printing first five items in the collection
for d in collection.find()[:].limit(5):
    print(d)

{'_id': ObjectId('5e8e71db86757736aa93aaa7'), 'MV_ID': 5502, 'MV_NAME': 'Mission: Impossible - Fallout', 'MV_RELDATE': 2018, 'MV_RUNTIME': '147 min', 'MV_RATING': 7.8, 'MV_DIRECTOR': 'Christopher McQuarrie', 'PC_NAME': 'Paramount Pictures'}
{'_id': ObjectId('5e8e71db86757736aa93aaa8'), 'MV_ID': 5504, 'MV_NAME': 'A Star Is Born', 'MV_RELDATE': 2018, 'MV_RUNTIME': '136 min', 'MV_RATING': 7.8, 'MV_DIRECTOR': 'Bradley Cooper', 'PC_NAME': 'Warner Bros. Pictures'}
{'_id': ObjectId('5e8e71db86757736aa93aaa9'), 'MV_ID': 5507, 'MV_NAME': 'You Were Never Really Here', 'MV_RELDATE': 2018, 'MV_RUNTIME': '89 min', 'MV_RATING': 6.8, 'MV_DIRECTOR': 'Lynne Ramsay', 'PC_NAME': 'Amazon Studios'}
{'_id': ObjectId('5e8e71db86757736aa93aaaa'), 'MV_ID': 5508, 'MV_NAME': 'A Quiet Place', 'MV_RELDATE': 2018, 'MV_RUNTIME': '90 min', 'MV_RATING': 7.6, 'MV_DIRECTOR': 'John Krasinski', 'PC_NAME': 'Paramount Pictures'}
{'_id': ObjectId('5e8e71db86757736aa93aaab'), 'MV_ID': 5509, 'MV_NAME': 'The Miseducation of Cam

In [15]:
collection = db.tweets



In [16]:
collection


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'assignment3'), 'tweets')

In [17]:
# READING CSV FILES INTO DATAFRAME

def load_csv(csv):
    p=os.path.join("data/", csv)
    print (p)
    data=pd.read_csv(p, encoding = "ISO-8859-1", engine='python')
    return data 
tweets_csv=load_csv('C:/Users/ashwi/PycharmProjects/dmddassignment3/ONFTweets.csv')
tweets_csv.head()

C:/Users/ashwi/PycharmProjects/dmddassignment3/ONFTweets.csv


,USR_ID,TW_ID,TW_DETAIL,TW_HASHTAG,MV_NAME
0,4228094679,21801,The Guilty (2018) - Read 15 Movie Reviews \nLi...,#TheGuilty,The Guilty
1,961325626304221000,21802,Has anybody been paying attention? @realDonald...,#TheGuilty,The Guilty
2,961325626304221000,21803,"?? Hmm, if the report was dropped this way so ...",#TheGuilty,The Guilty
3,70786076,21804,#TheGuilty\nSmall but very entertaining thrill...,#TheGuilty,The Guilty
4,961325626304221000,21805,AG Barr exonerated President Trump. Barr is no...,#TheGuilty,The Guilty


In [18]:
# the Tweets Dataframe file is converted into JSON and added into twts.

twts =json.loads(tweets_csv.to_json(orient='records'))
twts[0]


{'USR_ID': 4228094679,
 'TW_ID': 21801,
 'TW_DETAIL': 'The Guilty (2018) - Read 15 Movie Reviews \\nLink: https://t.co/ZbVWDatgRJ \\n#TheGuilty',
 'TW_HASHTAG': '#TheGuilty',
 'MV_NAME': 'The Guilty'}

In [19]:
try:
    result = db.tweets.drop()
    print ("testing movies drop")
except:
    pass



testing movies drop


In [20]:
# Inserting tweets JSON into the Tweets Collection

collection.insert(twts)

<ipython-input-20-e8893925c10c>:3: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  collection.insert(twts)


[ObjectId('5e8e71dc86757736aa93aad3'),
 ObjectId('5e8e71dc86757736aa93aad4'),
 ObjectId('5e8e71dc86757736aa93aad5'),
 ObjectId('5e8e71dc86757736aa93aad6'),
 ObjectId('5e8e71dc86757736aa93aad7'),
 ObjectId('5e8e71dc86757736aa93aad8'),
 ObjectId('5e8e71dc86757736aa93aad9'),
 ObjectId('5e8e71dc86757736aa93aada'),
 ObjectId('5e8e71dc86757736aa93aadb'),
 ObjectId('5e8e71dc86757736aa93aadc'),
 ObjectId('5e8e71dc86757736aa93aadd'),
 ObjectId('5e8e71dc86757736aa93aade'),
 ObjectId('5e8e71dc86757736aa93aadf'),
 ObjectId('5e8e71dc86757736aa93aae0'),
 ObjectId('5e8e71dc86757736aa93aae1'),
 ObjectId('5e8e71dc86757736aa93aae2'),
 ObjectId('5e8e71dc86757736aa93aae3'),
 ObjectId('5e8e71dc86757736aa93aae4'),
 ObjectId('5e8e71dc86757736aa93aae5'),
 ObjectId('5e8e71dc86757736aa93aae6'),
 ObjectId('5e8e71dc86757736aa93aae7'),
 ObjectId('5e8e71dc86757736aa93aae8'),
 ObjectId('5e8e71dc86757736aa93aae9'),
 ObjectId('5e8e71dc86757736aa93aaea'),
 ObjectId('5e8e71dc86757736aa93aaeb'),
 ObjectId('5e8e71dc867577

In [21]:

# Printing the first five tweets in the dataset

for d in db.tweets.find()[:].limit(5):
    print(d)


{'_id': ObjectId('5e8e71dc86757736aa93aad3'), 'USR_ID': 4228094679, 'TW_ID': 21801, 'TW_DETAIL': 'The Guilty (2018) - Read 15 Movie Reviews \\nLink: https://t.co/ZbVWDatgRJ \\n#TheGuilty', 'TW_HASHTAG': '#TheGuilty', 'MV_NAME': 'The Guilty'}
{'_id': ObjectId('5e8e71dc86757736aa93aad4'), 'USR_ID': 961325626304221000, 'TW_ID': 21802, 'TW_DETAIL': 'Has anybody been paying attention? @realDonaldTrump NEVER gives in or gives up. @POTUS Trump believes and eye for aÂ\x85 https://t.co/qZ8eRCYUbQ', 'TW_HASHTAG': '#TheGuilty', 'MV_NAME': 'The Guilty'}
{'_id': ObjectId('5e8e71dc86757736aa93aad5'), 'USR_ID': 961325626304221000, 'TW_ID': 21803, 'TW_DETAIL': '?? Hmm, if the report was dropped this way so #Trump could make The Dems Â\x93demand the reportÂ\x94.  They always take theÂ\x85 https://t.co/JcGVEqCrqe', 'TW_HASHTAG': '#TheGuilty', 'MV_NAME': 'The Guilty'}
{'_id': ObjectId('5e8e71dc86757736aa93aad6'), 'USR_ID': 70786076, 'TW_ID': 21804, 'TW_DETAIL': '#TheGuilty\\nSmall but very entertaining t

In [22]:
#TESTING

for d in db.tweets.find({'USR_ID': 4228094679}):
    print(d)

{'_id': ObjectId('5e8e71dc86757736aa93aad3'), 'USR_ID': 4228094679, 'TW_ID': 21801, 'TW_DETAIL': 'The Guilty (2018) - Read 15 Movie Reviews \\nLink: https://t.co/ZbVWDatgRJ \\n#TheGuilty', 'TW_HASHTAG': '#TheGuilty', 'MV_NAME': 'The Guilty'}
{'_id': ObjectId('5e8e71dc86757736aa93aad9'), 'USR_ID': 4228094679, 'TW_ID': 21807, 'TW_DETAIL': 'The Guilty (2018) - Read 15 Movie Reviews \\nLink: https://t.co/ZbVWDatgRJ \\n#TheGuilty', 'TW_HASHTAG': '#TheGuilty', 'MV_NAME': 'The Guilty'}
{'_id': ObjectId('5e8e71dc86757736aa93aaf0'), 'USR_ID': 4228094679, 'TW_ID': 21830, 'TW_DETAIL': 'Mission: Impossible - Fallout (2018) - Read 1,349 Movie Reviews \\nLink: https://t.co/sXZiwsQebV \\n#MissionImpossibleFallout', 'TW_HASHTAG': '#MissionImpossibleFallout', 'MV_NAME': 'Mission: Impossible - Fallout'}
{'_id': ObjectId('5e8e71dc86757736aa93ab35'), 'USR_ID': 4228094679, 'TW_ID': 21899, 'TW_DETAIL': 'Mission: Impossible - Fallout (2018) - Read 1,349 Movie Reviews \\nLink: https://t.co/sXZiwsQebV \\n#Miss

In [23]:
#  Question 1 What are tags are associated with a person, place or thing
# WHAT USERS ARE ASSOCIATED WITH A MOVIE
list(db.tweets.aggregate([
    {"$match":{"MV_NAME":"Mission: Impossible - Fallout"}},
    {"$group":{"_id":"$USR_ID","count":{"$sum":1}}},
    {"$sort":{"count":-1}}]))



[{'_id': 539165451, 'count': 3},
 {'_id': 1046996584288980000, 'count': 2},
 {'_id': 428846654, 'count': 2},
 {'_id': 4228094679, 'count': 2},
 {'_id': 1108377813323860000, 'count': 2},
 {'_id': 1110565597849570000, 'count': 2},
 {'_id': 66467134, 'count': 2},
 {'_id': 296950899, 'count': 1},
 {'_id': 704724513045487000, 'count': 1},
 {'_id': 2472522037, 'count': 1},
 {'_id': 222611753, 'count': 1},
 {'_id': 2159836623, 'count': 1},
 {'_id': 85818367, 'count': 1},
 {'_id': 1051525201094150000, 'count': 1},
 {'_id': 48328524, 'count': 1},
 {'_id': 19267296, 'count': 1},
 {'_id': 120177366, 'count': 1},
 {'_id': 611623330, 'count': 1},
 {'_id': 1050968042396570000, 'count': 1},
 {'_id': 243350702, 'count': 1},
 {'_id': 63797855, 'count': 1},
 {'_id': 2563460949, 'count': 1},
 {'_id': 3179238396, 'count': 1},
 {'_id': 152714268, 'count': 1},
 {'_id': 634833505, 'count': 1},
 {'_id': 976848548598870000, 'count': 1},
 {'_id': 810224647199490000, 'count': 1},
 {'_id': 767783292, 'count': 1},

In [24]:
# Question 2 What social media users are like other social media users in your domain?
# AGGREGATE ALL USER ID's WITH SIMILAR COMMENTS
match = {    'TW_DETAIL':{"$regex": "Mission Impossible"}
}

group = {
    '_id':"$USR_ID"
}

ret2 = db.tweets.aggregate([
    {'$match':match},
    {'$group': group}
])

list(ret2)


[{'_id': 354205351},
 {'_id': 810224647199490000},
 {'_id': 152714268},
 {'_id': 976848548598870000},
 {'_id': 428846654},
 {'_id': 36309760},
 {'_id': 767783292},
 {'_id': 23618134},
 {'_id': 2443006233},
 {'_id': 1108140293046570000},
 {'_id': 529850415},
 {'_id': 456074914},
 {'_id': 69730756},
 {'_id': 19899301},
 {'_id': 1464315997},
 {'_id': 139153610},
 {'_id': 416536319},
 {'_id': 194572083},
 {'_id': 2253589974},
 {'_id': 25855145},
 {'_id': 136992850},
 {'_id': 2243367863},
 {'_id': 491521774},
 {'_id': 296950899},
 {'_id': 704724513045487000},
 {'_id': 222611753},
 {'_id': 1919973978},
 {'_id': 946875617953353000},
 {'_id': 2159836623},
 {'_id': 20186972},
 {'_id': 48328524},
 {'_id': 611623330},
 {'_id': 2969341059},
 {'_id': 539165451},
 {'_id': 19267296},
 {'_id': 427305859},
 {'_id': 85818367},
 {'_id': 302702394},
 {'_id': 270819857},
 {'_id': 1116159427},
 {'_id': 2563460949},
 {'_id': 2231267989},
 {'_id': 634833505},
 {'_id': 351619045}]

In [25]:
# Question  3 What people, places or things are popular in your domain
#  WHICH MOVIES HAVE THE MOST NUMBER OF TWEETS - DESCENDING
list(db.tweets.aggregate([
    {"$group":{"_id":"$MV_NAME","count":{"$sum":1}}},
    {"$sort":{"count":-1}}]))

[{'_id': 'The Marvelous Mrs. Maisel', 'count': 100},
 {'_id': 'Game Night', 'count': 100},
 {'_id': 'Deadpool 2', 'count': 100},
 {'_id': 'Ready Player One', 'count': 100},
 {'_id': 'The First Purge', 'count': 100},
 {'_id': 'Ozark', 'count': 100},
 {'_id': 'UnREAL', 'count': 100},
 {'_id': 'Eighth Grade', 'count': 100},
 {'_id': 'Ant-Man and the Wasp', 'count': 100},
 {'_id': 'The Nun', 'count': 100},
 {'_id': 'Daredevil', 'count': 100},
 {'_id': 'Roma', 'count': 100},
 {'_id': 'Titans', 'count': 100},
 {'_id': 'Rampage', 'count': 100},
 {'_id': 'Sharp Objects', 'count': 100},
 {'_id': 'Solo: A Star Wars Story', 'count': 100},
 {'_id': 'Free Solo', 'count': 100},
 {'_id': 'Jack Ryan', 'count': 100},
 {'_id': 'Widows', 'count': 100},
 {'_id': 'Upgrade', 'count': 100},
 {'_id': 'Annihilation', 'count': 100},
 {'_id': 'Apostle', 'count': 100},
 {'_id': 'Searching', 'count': 100},
 {'_id': 'Gringo', 'count': 100},
 {'_id': 'The Grinch', 'count': 100},
 {'_id': 'The Meg', 'count': 99},
 {'

In [26]:
# question 4 What people, places or things are trending in your domain? (A trend is popularity over time.)
# WHICH PRODUCTION COMPANIES ARE TRENDING
list(db.movies.aggregate([
    {"$sort":{"MV_RELDATE":1}}]))

for doc in db.movies.find({},{"PC_NAME":1, "_id":0, "MV_RELDATE":1}).sort('MV_RELDATE', pymongo.ASCENDING):
    
    print(doc)




{'MV_RELDATE': 2018, 'PC_NAME': 'Paramount Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Warner Bros. Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Amazon Studios'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Paramount Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'FilmRise'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Greenwich Entertainment'}
{'MV_RELDATE': 2018, 'PC_NAME': '20th Century Fox'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Walt Disney Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Focus Features'}
{'MV_RELDATE': 2018, 'PC_NAME': 'NEON'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Disney/Pixar'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Netflix'}
{'MV_RELDATE': 2018, 'PC_NAME': 'NEON'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Lionsgate and Roadside Attractions'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Magnolia Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Warner Bros. Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Universal Pictures'}
{'MV_RELDATE': 2018, 'PC_NAME': 'New Line Cinema'}
{'MV_RELDATE': 2018, 'PC_NAME': 'Paramount Pictures'}
{'MV_RELDATE':

In [27]:
# USE CASES

# 1. Who directed this movie?
for d in db.movies.find({'MV_NAME':'Deadpool 2'},{'_id':0, 'MV_ID':0,  'MV_RUNTIME':0,'MV_RATING':0,'PC_NAME':0}):
    print(d)


{'MV_NAME': 'Deadpool 2', 'MV_RELDATE': 2018, 'MV_DIRECTOR': 'David Leitch'}


In [28]:
# 2. Which movie has highest rating?

for doc in db.movies.find({},{"MV_ID":0, "_id":0, "MV_RELDATE":0, "MV_RUNTIME":0, "MV_DIRECTOR":0, "PC_NAME":0,}).sort('MV_RATING', pymongo.DESCENDING):
    
    print(doc)



{'MV_NAME': 'Daredevil', 'MV_RATING': 8.7}
{'MV_NAME': 'Avengers: Infinity War', 'MV_RATING': 8.5}
{'MV_NAME': 'Free Solo', 'MV_RATING': 8.4}
{'MV_NAME': 'Jack Ryan', 'MV_RATING': 8.2}
{'MV_NAME': 'UnREAL', 'MV_RATING': 7.9}
{'MV_NAME': 'Mission: Impossible - Fallout', 'MV_RATING': 7.8}
{'MV_NAME': 'A Star Is Born', 'MV_RATING': 7.8}
{'MV_NAME': 'Deadpool 2', 'MV_RATING': 7.8}
{'MV_NAME': 'Incredibles 2', 'MV_RATING': 7.8}
{'MV_NAME': 'Love, Simon', 'MV_RATING': 7.7}
{'MV_NAME': 'A Quiet Place', 'MV_RATING': 7.6}
{'MV_NAME': 'BlacKkKlansman', 'MV_RATING': 7.5}
{'MV_NAME': 'RBG', 'MV_RATING': 7.5}
{'MV_NAME': 'Ready Player One', 'MV_RATING': 7.5}
{'MV_NAME': 'First Man', 'MV_RATING': 7.4}
{'MV_NAME': 'Creed II', 'MV_RATING': 7.4}
{'MV_NAME': 'Hereditary', 'MV_RATING': 7.3}
{'MV_NAME': 'Aquaman', 'MV_RATING': 7.3}
{'MV_NAME': 'Ralph Breaks the Internet', 'MV_RATING': 7.2}
{'MV_NAME': 'Bumblebee', 'MV_RATING': 7.1}
{'MV_NAME': 'Bad Times at the El Royale', 'MV_RATING': 7.1}
{'MV_NAME': 'G

In [29]:
# 3. Who is the most active User?

list(db.tweets.aggregate([{"$group":{"_id":"$USR_ID","count":{"$sum":1}}},{"$sort":{"count":-1}}]))

[{'_id': 551331049, 'count': 68},
 {'_id': 4228094679, 'count': 61},
 {'_id': 112764063, 'count': 46},
 {'_id': 2599216440, 'count': 28},
 {'_id': 943905626920402000, 'count': 27},
 {'_id': 32608699, 'count': 23},
 {'_id': 340314799, 'count': 21},
 {'_id': 1063066483776020000, 'count': 19},
 {'_id': 3435780819, 'count': 18},
 {'_id': 763906257409314000, 'count': 17},
 {'_id': 397921436, 'count': 16},
 {'_id': 1548271734, 'count': 14},
 {'_id': 777834278566658000, 'count': 13},
 {'_id': 556636611, 'count': 13},
 {'_id': 604647998, 'count': 13},
 {'_id': 135841986, 'count': 12},
 {'_id': 68777692, 'count': 11},
 {'_id': 118486986, 'count': 10},
 {'_id': 147223992, 'count': 9},
 {'_id': 215295837, 'count': 9},
 {'_id': 2828723942, 'count': 8},
 {'_id': 1105447223591120000, 'count': 8},
 {'_id': 1109702278632680000, 'count': 8},
 {'_id': 1095857065040460000, 'count': 8},
 {'_id': 18232843, 'count': 8},
 {'_id': 978973907544952000, 'count': 8},
 {'_id': 468733563, 'count': 7},
 {'_id': 1081